In [40]:
import pandas as pd

#read file and visualise it

data = pd.read_csv('dataset.csv', nrows=300000)

In [41]:
print(data)

        app_id                    app_name  \
0           10              Counter-Strike   
1           10              Counter-Strike   
2           10              Counter-Strike   
3           10              Counter-Strike   
4           10              Counter-Strike   
...        ...                         ...   
299995   12200  Bully: Scholarship Edition   
299996   12200  Bully: Scholarship Edition   
299997   12200  Bully: Scholarship Edition   
299998   12200  Bully: Scholarship Edition   
299999   12200  Bully: Scholarship Edition   

                                              review_text  review_score  \
0                                         Ruined my life.             1   
1       This will be more of a ''my experience with th...             1   
2                           This game saved my virginity.             1   
3       • Do you like original games? • Do you like ga...             1   
4                Easy to learn, hard to master.                       1 

In [44]:
#Clean the data:
import re
#Remove duplicates, null values, and irrelevant information.
#Handle noisy data (e.g., special characters, HTML tags).
data.drop_duplicates()
#print(data['review_text'])

for review in data['review_text']:
    review = re.sub('\W+',' ', str(review) ).lower()



In [47]:
data['review_text'].dropna(inplace=True)

#Text preprocessing:
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#Tokenization, lowercasing, removing stopwords, stemming, and lemmatization.
text = data['review_text'].str.lower().astype('str')

text = text.apply(word_tokenize)
text.head()

0                                [ruined, my, life, .]
1    [this, will, be, more, of, a, ``, my, experien...
2                [this, game, saved, my, virginity, .]
3    [•, do, you, like, original, games, ?, •, do, ...
4            [easy, to, learn, ,, hard, to, master, .]
Name: review_text, dtype: object

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
filtered_sentences = []
for texts in text:
    filtered_sentence = []
    for word in texts:
        if word not in stop_words:
            filtered_sentence.append(word)
    filtered_sentences.append(filtered_sentence)
print(len(filtered_sentences))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bobby\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
